# Complete CBM Pipeline - Concept Bottleneck Model

This notebook implements a complete pipeline for:
1. Loading preprocessed data from disk
2. Training a simple Concept Bottleneck Model (CBM)
3. Evaluating with detailed metrics and concept probabilities

**Architecture:** X → Concept Logits → Concepts → Task Prediction

## Section 0: Configuration & Setup

In [1]:
# Imports
import os
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    roc_auc_score,
    balanced_accuracy_score,
    classification_report,
)

print("✓ All imports successful")

✓ All imports successful


In [2]:
# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
pl.seed_everything(SEED)

print(f"✓ Random seed set to {SEED}")

Global seed set to 42


✓ Random seed set to 42


In [3]:
# Detect device (MPS/CUDA/CPU)
if torch.backends.mps.is_available():
    DEVICE = "mps"
    print("✓ Using MacBook GPU (MPS)")
elif torch.cuda.is_available():
    DEVICE = "cuda"
    print("✓ Using CUDA GPU")
else:
    DEVICE = "cpu"
    print("⚠ Using CPU (will be slow)")

✓ Using MacBook GPU (MPS)


In [4]:
# Define paths
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data/processed")
DATASET_DIR = os.path.join(DATA_PROCESSED, "whole_pipeline")
OUTPUT_DIR = "outputs_cbm"

print("✓ Paths configured")
print(f"  Project root: {PROJECT_ROOT}")
print(f"  Dataset dir: {DATASET_DIR}")
print(f"  Output dir: {OUTPUT_DIR}")

✓ Paths configured
  Project root: /Users/gualtieromarencoturi/Desktop/thesis/Master-Thesis-CEM-Depression-etc-case-study
  Dataset dir: /Users/gualtieromarencoturi/Desktop/thesis/Master-Thesis-CEM-Depression-etc-case-study/data/processed/whole_pipeline
  Output dir: outputs_cbm


In [5]:
# Define 21 BDI-II concept names
CONCEPT_NAMES = [
    "Sadness", "Pessimism", "Past failure", "Loss of pleasure",
    "Guilty feelings", "Punishment feelings", "Self-dislike", "Self-criticalness",
    "Suicidal thoughts or wishes", "Crying", "Agitation", "Loss of interest",
    "Indecisiveness", "Worthlessness", "Loss of energy", "Changes in sleeping pattern",
    "Irritability", "Changes in appetite", "Concentration difficulty",
    "Tiredness or fatigue", "Loss of interest in sex"
]
N_CONCEPTS = len(CONCEPT_NAMES)

print(f"✓ Defined {N_CONCEPTS} BDI-II concepts")

✓ Defined 21 BDI-II concepts


In [6]:
# Hyperparameters
HYPERPARAMS = {
    # Model architecture
    "embedding_dim": 384,       # SBERT embedding dimension
    "n_concepts": 21,
    "n_tasks": 1,
    
    # Training
    "batch_size_train": 32,
    "batch_size_eval": 64,
    "max_epochs": 100,
    "learning_rate": 0.001,     # Lower LR for AdamW
    "weight_decay": 0.01,
    
    # Loss weights
    "concept_loss_weight": 1.0,
}

print("✓ Hyperparameters configured:")
for k, v in HYPERPARAMS.items():
    print(f"  {k}: {v}")

✓ Hyperparameters configured:
  embedding_dim: 384
  n_concepts: 21
  n_tasks: 1
  batch_size_train: 32
  batch_size_eval: 64
  max_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.01
  concept_loss_weight: 1.0


## Section 1: Load Preprocessed Data

Load the datasets that were saved by the CEM pipeline

In [7]:
# Load training data
print("Loading preprocessed datasets...")

train_data = np.load(os.path.join(DATASET_DIR, "train_data.npz"))
X_train = train_data['X']
C_train = train_data['C']
y_train = train_data['y']
train_subject_ids = train_data['subject_ids']

print(f"✓ Loaded training data:")
print(f"  X_train: {X_train.shape}")
print(f"  C_train: {C_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  Subject IDs: {len(train_subject_ids)}")

Loading preprocessed datasets...
✓ Loaded training data:
  X_train: (486, 384)
  C_train: (486, 21)
  y_train: (486,)
  Subject IDs: 486


In [8]:
# Load validation data
val_data = np.load(os.path.join(DATASET_DIR, "val_data.npz"))
X_val = val_data['X']
C_val = val_data['C']
y_val = val_data['y']
val_subject_ids = val_data['subject_ids']

print(f"✓ Loaded validation data:")
print(f"  X_val: {X_val.shape}")
print(f"  C_val: {C_val.shape}")
print(f"  y_val: {y_val.shape}")

✓ Loaded validation data:
  X_val: (200, 384)
  C_val: (200, 21)
  y_val: (200,)


In [9]:
# Load test data
test_data = np.load(os.path.join(DATASET_DIR, "test_data.npz"))
X_test = test_data['X']
C_test = test_data['C']
y_test = test_data['y']
test_subject_ids = test_data['subject_ids']

print(f"✓ Loaded test data:")
print(f"  X_test: {X_test.shape}")
print(f"  C_test: {C_test.shape}")
print(f"  y_test: {y_test.shape}")

✓ Loaded test data:
  X_test: (201, 384)
  C_test: (201, 21)
  y_test: (201,)


In [10]:
# Load class weights
with open(os.path.join(DATASET_DIR, "class_weights.json"), 'r') as f:
    class_info = json.load(f)

n_positive = class_info['n_positive']
n_negative = class_info['n_negative']
pos_weight = class_info['pos_weight']

# Convert to tensor
pos_weight_tensor = torch.tensor([pos_weight], dtype=torch.float32)

print(f"✓ Loaded class weights:")
print(f"  Negative samples: {n_negative}")
print(f"  Positive samples: {n_positive}")
print(f"  Ratio: 1:{pos_weight:.2f}")
print(f"  pos_weight: {pos_weight:.4f}")

✓ Loaded class weights:
  Negative samples: 403
  Positive samples: 83
  Ratio: 1:4.86
  pos_weight: 4.8554


## Section 2: PyTorch Dataset & DataLoaders

In [11]:
class CBMDataset(Dataset):
    """PyTorch Dataset for CBM model."""
    
    def __init__(self, X, C, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.C = torch.tensor(C, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.C[idx]

print("✓ CBMDataset class defined")

✓ CBMDataset class defined


In [12]:
# Create datasets
train_dataset = CBMDataset(X_train, C_train, y_train)
val_dataset = CBMDataset(X_val, C_val, y_val)
test_dataset = CBMDataset(X_test, C_test, y_test)

print("✓ Datasets created")
print(f"  Train: {len(train_dataset)} samples")
print(f"  Val: {len(val_dataset)} samples")
print(f"  Test: {len(test_dataset)} samples")

✓ Datasets created
  Train: 486 samples
  Val: 200 samples
  Test: 201 samples


In [13]:
# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=HYPERPARAMS['batch_size_train'],
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=HYPERPARAMS['batch_size_eval'],
    shuffle=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=HYPERPARAMS['batch_size_eval'],
    shuffle=False
)

print("✓ DataLoaders created")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

# Test batch
x_batch, y_batch, c_batch = next(iter(train_loader))
print(f"\n  Sample batch shapes:")
print(f"    X: {x_batch.shape}")
print(f"    y: {y_batch.shape}")
print(f"    C: {c_batch.shape}")

✓ DataLoaders created
  Train batches: 16
  Val batches: 4
  Test batches: 4

  Sample batch shapes:
    X: torch.Size([32, 384])
    y: torch.Size([32])
    C: torch.Size([32, 21])


## Section 3: Concept Bottleneck Model

Simple CBM architecture:
- X → Concept Extractor → Concept Logits
- Concept Probabilities (sigmoid) → Task Classifier → Task Logits

In [14]:
class ConceptBottleneckModel(pl.LightningModule):
    """
    Minimal concept bottleneck model.
    Architecture:
      X -> concept extractor (logits) -> sigmoid -> predicted concepts
      predicted concepts -> simple classifier -> task logits
    Training loss:
      task_loss = BCEWithLogitsLoss(pos_weight=pos_weight)
      concept_loss = BCEWithLogitsLoss (computed only when concept labels provided in batch)
      total_loss = task_loss + concept_loss_weight * concept_loss
    """
    def __init__(
        self,
        input_dim,
        n_concepts,
        task_output_dim=1,
        c_extractor_arch=None,
        learning_rate=1e-3,
        weight_decay=0.0,
        concept_loss_weight=1.0,
        pos_weight_tensor=None,
    ):
        super().__init__()
        self.save_hyperparameters()

        # Concept extractor: if user provided a callable, use it to build final layer
        if c_extractor_arch is None:
            # simple two-layer MLP
            self.c_extractor = nn.Sequential(
                nn.Linear(input_dim, 256),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(256, n_concepts),
            )
        else:
            # expect c_extractor_arch to be a callable that returns nn.Module when given output_dim
            self.c_extractor = c_extractor_arch(n_concepts)

        # classifier from predicted concepts to task logits
        self.c2y = nn.Sequential(
            nn.Linear(n_concepts, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, task_output_dim)
        )

        # Loss functions
        # concept loss: per-concept BCE
        self.concept_loss_fn = nn.BCEWithLogitsLoss()
        # task loss: allow pos_weight for imbalanced binary classification
        self.pos_weight_tensor = pos_weight_tensor
        if pos_weight_tensor is not None:
            self.task_loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
        else:
            self.task_loss_fn = nn.BCEWithLogitsLoss()

        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.concept_loss_weight = concept_loss_weight

    def forward(self, x):
        # x: (B, input_dim)
        c_logits = self.c_extractor(x)  # (B, n_concepts)
        # we'll produce y_logits from concept probabilities
        c_probs = torch.sigmoid(c_logits)
        y_logits = self.c2y(c_probs)  # (B, task_output_dim)
        return c_logits, y_logits

    def training_step(self, batch, batch_idx):
        x, y, c_true = batch
        c_logits, y_logits = self.forward(x)

        # task loss
        y = y.view_as(y_logits).float()
        task_loss = self.task_loss_fn(y_logits, y)

        # concept loss, only when concept ground truth labels exist in the batch
        concept_loss = torch.tensor(0.0, device=self.device)
        # detect if any label is provided (non all zeros and not all missing)
        if c_true.numel() > 0 and torch.sum(c_true) > 0:
            concept_loss = self.concept_loss_fn(c_logits, c_true)

        loss = task_loss + self.concept_loss_weight * concept_loss

        # logging
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.log('train_task_loss', task_loss, on_step=False, on_epoch=True)
        self.log('train_concept_loss', concept_loss, on_step=False, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y, c_true = batch
        c_logits, y_logits = self.forward(x)

        y_prob = torch.sigmoid(y_logits).detach()
        y_pred = (y_prob >= 0.5).int()

        # compute losses similarly as training (concept loss only if labels present)
        y = y.view_as(y_logits).float()
        task_loss = self.task_loss_fn(y_logits, y)
        concept_loss = torch.tensor(0.0, device=self.device)
        if c_true.numel() > 0 and torch.sum(c_true) > 0:
            concept_loss = self.concept_loss_fn(c_logits, c_true)
        loss = task_loss + self.concept_loss_weight * concept_loss

        # return preds for metric aggregation
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        return {
            'val_loss': loss,
            'y_true': y.detach().cpu(),
            'y_prob': y_prob.detach().cpu(),
        }

    def test_step(self, batch, batch_idx):
        x, y, c_true = batch
        c_logits, y_logits = self.forward(x)
        c_probs = torch.sigmoid(c_logits)
        y_probs = torch.sigmoid(y_logits).squeeze(-1)
        return {
            'y_true': y.detach().cpu(),
            'y_probs': y_probs.detach().cpu(),
            'c_probs': c_probs.detach().cpu(),
        }

    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        return opt

print("✓ ConceptBottleneckModel class defined")

✓ ConceptBottleneckModel class defined


## Section 4: Model Initialization

In [15]:
def c_extractor_arch(output_dim):
    """Concept extractor architecture."""
    return nn.Sequential(
        nn.Linear(HYPERPARAMS['embedding_dim'], 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, output_dim)
    )

print("✓ Concept extractor architecture defined")

✓ Concept extractor architecture defined


In [16]:
# Initialize CBM model
cbm_model = ConceptBottleneckModel(
    input_dim=HYPERPARAMS['embedding_dim'],
    n_concepts=HYPERPARAMS['n_concepts'],
    task_output_dim=1,
    c_extractor_arch=c_extractor_arch,
    learning_rate=HYPERPARAMS['learning_rate'],
    weight_decay=HYPERPARAMS['weight_decay'],
    concept_loss_weight=HYPERPARAMS['concept_loss_weight'],
    pos_weight_tensor=pos_weight_tensor,  # Use class weights for imbalanced data
)

print("✓ CBM model initialized")
print(f"  Using pos_weight={pos_weight:.4f} for class imbalance")
print(cbm_model)

✓ CBM model initialized
  Using pos_weight=4.8554 for class imbalance
ConceptBottleneckModel(
  (c_extractor): Sequential(
    (0): Linear(in_features=384, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=21, bias=True)
  )
  (c2y): Sequential(
    (0): Linear(in_features=21, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
  (concept_loss_fn): BCEWithLogitsLoss()
  (task_loss_fn): BCEWithLogitsLoss()
)


## Section 5: Training

In [17]:
# Setup trainer
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath=os.path.join(OUTPUT_DIR, "models"),
    filename="cbm-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
    mode="min"
)

trainer = pl.Trainer(
    max_epochs=HYPERPARAMS['max_epochs'],
    accelerator=DEVICE,
    devices=1,
    logger=CSVLogger(save_dir=os.path.join(OUTPUT_DIR, "logs"), name="cbm_pipeline"),
    log_every_n_steps=10,
    callbacks=[checkpoint_callback],
    enable_progress_bar=True
)

print("✓ Trainer configured")
print(f"  Device: {DEVICE}")
print(f"  Max epochs: {HYPERPARAMS['max_epochs']}")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


✓ Trainer configured
  Device: mps
  Max epochs: 100


In [18]:
# Train model
print("\nStarting training...\n")
print("="*70)

trainer.fit(cbm_model, train_loader, val_loader)

print("="*70)
print("\n✓ Training complete!")

/opt/miniconda3/envs/38concept_embedding/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory outputs_cbm/models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name            | Type              | Params
------------------------------------------------------
0 | c_extractor     | Sequential        | 103 K 
1 | c2y             | Sequential        | 1.5 K 
2 | concept_loss_fn | BCEWithLogitsLoss | 0     
3 | task_loss_fn    | BCEWithLogitsLoss | 0     
------------------------------------------------------
105 K     Trainable params
0         Non-trainable params
105 K     Total params
0.422     Total estimated model params size (MB)



Starting training...



Sanity Checking: 0it [00:00, ?it/s]

/opt/miniconda3/envs/38concept_embedding/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/miniconda3/envs/38concept_embedding/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.



✓ Training complete!


## Section 6: Test Evaluation

In [19]:
# Set model to evaluation mode
cbm_model.eval()

# Move model to device
device_obj = torch.device(DEVICE)
cbm_model = cbm_model.to(device_obj)

print("✓ Model set to evaluation mode")

✓ Model set to evaluation mode


In [20]:
# Run inference on test set
print("Running inference on test set...")

y_true_list = []
y_pred_list = []
y_prob_list = []
concept_probs_list = []

with torch.no_grad():
    for x_batch, y_batch, c_batch in test_loader:
        x_batch = x_batch.to(device_obj)
        
        # Forward pass
        c_logits, y_logits = cbm_model(x_batch)
        
        # Apply sigmoid to get probabilities
        c_probs = torch.sigmoid(c_logits).cpu().numpy()
        y_probs = torch.sigmoid(y_logits).cpu().squeeze().numpy()
        
        # Threshold at 0.5 for predictions
        y_pred = (y_probs >= 0.5).astype(int)
        
        # Collect results
        y_true_list.extend(y_batch.numpy().astype(int).tolist())
        y_pred_list.extend(y_pred.tolist() if isinstance(y_pred, np.ndarray) else [y_pred])
        y_prob_list.extend(y_probs.tolist() if isinstance(y_probs, np.ndarray) else [y_probs])
        concept_probs_list.extend(c_probs.tolist())

# Convert to arrays
y_true = np.array(y_true_list)
y_pred = np.array(y_pred_list)
y_prob = np.array(y_prob_list)
concept_probs = np.array(concept_probs_list)

print("✓ Inference complete")
print(f"  Predictions shape: {y_pred.shape}")
print(f"  Concept probs shape: {concept_probs.shape}")

Running inference on test set...
✓ Inference complete
  Predictions shape: (201,)
  Concept probs shape: (201, 21)


## Section 7: Metrics & Results Display

In [21]:
# Compute all metrics
print("Computing metrics...")

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

# Metrics
acc = accuracy_score(y_true, y_pred)
balanced_acc = balanced_accuracy_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_prob)
mcc = matthews_corrcoef(y_true, y_pred)

f1_binary = f1_score(y_true, y_pred, pos_label=1)
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')

precision_binary = precision_score(y_true, y_pred, pos_label=1)
recall_binary = recall_score(y_true, y_pred, pos_label=1)

print("✓ Metrics computed")

Computing metrics...
✓ Metrics computed


In [22]:
# Print formatted results
print("\n" + "="*70)
print("                    TEST SET EVALUATION")
print("="*70)
print()
print(f"Dataset Statistics:")
print(f"  Test subjects:        {len(y_true)}")
print(f"  Positive cases:       {np.sum(y_true)} ({100*np.sum(y_true)/len(y_true):.1f}%)")
print(f"  Negative cases:       {len(y_true)-np.sum(y_true)} ({100*(len(y_true)-np.sum(y_true))/len(y_true):.1f}%)")
print()
print(f"Performance Metrics:")
print(f"  Accuracy:                  {acc:.4f}")
print(f"  Balanced Accuracy:         {balanced_acc:.4f}")
print(f"  ROC-AUC:                   {roc_auc:.4f}")
print(f"  Matthews Correlation:      {mcc:.4f}")
print()
print(f"  F1 Score (Binary):         {f1_binary:.4f}")
print(f"  F1 Score (Macro):          {f1_macro:.4f}")
print(f"  F1 Score (Micro):          {f1_micro:.4f}")
print()
print(f"  Precision (Binary):        {precision_binary:.4f}")
print(f"  Recall (Binary):           {recall_binary:.4f}")
print()
print(f"Confusion Matrix:")
print(f"                    Predicted Neg    Predicted Pos")
print(f"Actual Neg          {tn:^16d} {fp:^16d}")
print(f"Actual Pos          {fn:^16d} {tp:^16d}")
print()
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Negative', 'Positive']))
print("="*70)


                    TEST SET EVALUATION

Dataset Statistics:
  Test subjects:        201
  Positive cases:       26 (12.9%)
  Negative cases:       175 (87.1%)

Performance Metrics:
  Accuracy:                  0.8806
  Balanced Accuracy:         0.7513
  ROC-AUC:                   0.8620
  Matthews Correlation:      0.4871

  F1 Score (Binary):         0.5556
  F1 Score (Macro):          0.7433
  F1 Score (Micro):          0.8806

  Precision (Binary):        0.5357
  Recall (Binary):           0.5769

Confusion Matrix:
                    Predicted Neg    Predicted Pos
Actual Neg                162               13       
Actual Pos                 11               15       

Classification Report:
              precision    recall  f1-score   support

    Negative       0.94      0.93      0.93       175
    Positive       0.54      0.58      0.56        26

    accuracy                           0.88       201
   macro avg       0.74      0.75      0.74       201
weighted avg     

In [23]:
# Save metrics to JSON
metrics_dict = {
    "n_samples": int(len(y_true)),
    "n_positive": int(np.sum(y_true)),
    "n_negative": int(len(y_true) - np.sum(y_true)),
    "accuracy": float(acc),
    "balanced_accuracy": float(balanced_acc),
    "roc_auc": float(roc_auc),
    "mcc": float(mcc),
    "f1_binary": float(f1_binary),
    "f1_macro": float(f1_macro),
    "f1_micro": float(f1_micro),
    "precision_binary": float(precision_binary),
    "recall_binary": float(recall_binary),
    "confusion_matrix": {
        "tn": int(tn),
        "fp": int(fp),
        "fn": int(fn),
        "tp": int(tp)
    }
}

os.makedirs(os.path.join(OUTPUT_DIR, "results"), exist_ok=True)
metrics_path = os.path.join(OUTPUT_DIR, "results/test_metrics.json")

with open(metrics_path, 'w') as f:
    json.dump(metrics_dict, f, indent=4)

print(f"✓ Metrics saved to {metrics_path}")

✓ Metrics saved to outputs_cbm/results/test_metrics.json


In [24]:
# Create predictions DataFrame with concept probabilities
predictions_df = pd.DataFrame({
    'subject_id': test_subject_ids,
    'y_true': y_true,
    'y_pred': y_pred,
    'y_prob': y_prob
})

# Add concept probabilities
for i, concept_name in enumerate(CONCEPT_NAMES):
    predictions_df[concept_name] = concept_probs[:, i]

# Save to CSV
predictions_path = os.path.join(OUTPUT_DIR, "results/test_predictions.csv")
predictions_df.to_csv(predictions_path, index=False)

print(f"✓ Predictions saved to {predictions_path}")
print(f"\nFirst 10 subjects with concept probabilities:")
print(predictions_df.head(10))

✓ Predictions saved to outputs_cbm/results/test_predictions.csv

First 10 subjects with concept probabilities:
         subject_id  y_true  y_pred    y_prob   Sadness  Pessimism  \
0  test_subject4471       1       0  0.057161  0.039895   0.168194   
1  test_subject8981       0       0  0.033149  0.033102   0.125926   
2  test_subject8777       0       0  0.007389  0.001647   0.011800   
3  test_subject1372       0       0  0.010126  0.013687   0.067205   
4  test_subject1830       0       0  0.073305  0.043946   0.153163   
5  test_subject3791       0       0  0.006998  0.000253   0.002527   
6  test_subject2284       0       0  0.008166  0.004009   0.026668   
7  test_subject5689       0       0  0.008905  0.007190   0.038363   
8  test_subject7467       1       0  0.010271  0.011724   0.062313   
9  test_subject7578       0       0  0.007090  0.000518   0.004261   

   Past failure  Loss of pleasure  Guilty feelings  Punishment feelings  ...  \
0      0.144256          0.046730     

In [25]:
# Display concept activation statistics
print("\nConcept Activation Statistics:")
print("="*70)
print(f"{'Concept':<35} {'Mean':>10} {'Std':>10} {'Max':>10}")
print("-"*70)
for i, concept_name in enumerate(CONCEPT_NAMES):
    mean_act = np.mean(concept_probs[:, i])
    std_act = np.std(concept_probs[:, i])
    max_act = np.max(concept_probs[:, i])
    print(f"{concept_name:<35} {mean_act:>10.4f} {std_act:>10.4f} {max_act:>10.4f}")
print("="*70)


Concept Activation Statistics:
Concept                                   Mean        Std        Max
----------------------------------------------------------------------
Sadness                                 0.0383     0.0733     0.4175
Pessimism                               0.1040     0.1490     0.6800
Past failure                            0.0985     0.1527     0.7107
Loss of pleasure                        0.0417     0.0751     0.3897
Guilty feelings                         0.0239     0.0499     0.3064
Punishment feelings                     0.0564     0.0894     0.4342
Self-dislike                            0.1752     0.2294     0.8907
Self-criticalness                       0.0495     0.0864     0.4493
Suicidal thoughts or wishes             0.0236     0.0428     0.2456
Crying                                  0.0256     0.0440     0.2557
Agitation                               0.0101     0.0183     0.1094
Loss of interest                        0.0577     0.0976     0.4843


## Section 8: Summary

In [26]:
print("\n" + "="*70)
print("              CBM PIPELINE EXECUTION COMPLETE")
print("="*70)
print("\nGenerated files:")
print(f"  Model checkpoint: {OUTPUT_DIR}/models/")
print(f"  Metrics JSON:     {OUTPUT_DIR}/results/test_metrics.json")
print(f"  Predictions CSV:  {OUTPUT_DIR}/results/test_predictions.csv")
print(f"  Training logs:    {OUTPUT_DIR}/logs/")
print("="*70)


              CBM PIPELINE EXECUTION COMPLETE

Generated files:
  Model checkpoint: outputs_cbm/models/
  Metrics JSON:     outputs_cbm/results/test_metrics.json
  Predictions CSV:  outputs_cbm/results/test_predictions.csv
  Training logs:    outputs_cbm/logs/
